In [1]:
import configparser
import unittest
import datetime
import sys
import pandas as pd

# from tqdm import tqdm_notebook

sys.path.append("/home/sergey/drclinics/common")
from universal_connection import UniversalConnection, DBType

sys.path.append("/home/sergey/drclinics/reports")
# from report_utils import excel_report

import numpy as np
from log import log

In [55]:
import datetime

In [79]:
from dateutil.relativedelta import relativedelta

In [2]:
sql = """
select 	promo.id promo_id,
		prod.full_name product_name,
		--patient.id patient_id,
		--date(promo.start_ + 3 * interval '1 hour') date_start,
		--date(promo.end_ + 3 * interval '1 hour' - interval '1 year') date_start_end,
		case 
        	when date(promo.start_ + 3 * interval '1 hour') < '2016-01-01' then date(promo.end_ + 3 * interval '1 hour' - interval '1 year')
        	else date(promo.start_ + 3 * interval '1 hour')
       	end normal_start,
		--date_part('year', promo.start_ + 3 * interval '1 hour') start_year,
		--date_part('month', promo.start_ + 3 * interval '1 hour') start_month,
		date(promo.end_ + 3 * interval '1 hour') date_end,
		--date_part('year', promo.end_ + 3 * interval '1 hour') start_year,
		--date_part('month', promo.end_ + 3 * interval '1 hour') start_month,
		a.id app_id,
		date(a.finished + 3 * interval '1 hour') app_finished--,
		--date_part('year', a.finished + 3 * interval '1 hour') app_finished_year,
		--date_part('month', a.finished + 3 * interval '1 hour') app_finished_month
from promotion promo
inner join product_condition pc
   on promo.product_condition_id = pc.id
inner join product prod
   on prod.id = pc.product_id
----------------
inner join appointment a on promo.id = a.promotion_id 
where lower(prod.full_name) not like '%подписка%' 
and a.good
  and not (
    a.patient_id in (
        select patient_id
        from patient_categories pcat
        inner join reference rf
           on rf.id=pcat.reference_id
        where rf.code='TEST'
        )
    or lower(a.report_comment)='тест'
    )
and date(a.finished + 3 * interval '1 hour') is not null
--and date(promo.start_ + 3 * interval '1 hour') < '1990-01-01'
order by promo.id
"""

In [3]:
connection = UniversalConnection('../../../.credentials/telemed/prom.cfg', DBType.Postgres)
data = connection.query(sql)
connection.close()

2020-05-08 11:53:13 connect to postgres database using config file "../../../.credentials/telemed/prom.cfg"
2020-05-08 11:53:13 creating ssh tunnel to 172.16.100.19 as root...
2020-05-08 11:53:14 connect postgres using parameters:
                    database: telemed
				    user: norekhov
				    password: ***masked***
				    host: localhost
				    port: 33711
2020-05-08 11:53:14 @telemed: execute sql:
				    SET TIME ZONE 'Europe/Moscow'
				    None
2020-05-08 11:53:14 @telemed query:
                    select 	promo.id promo_id,
				    		prod.full_name product_name,
				    		--patient.id patient_id,
				    		--date(promo.start_ + 3 * interval '1 hour') date_start,
				    		--date(promo.end_ + 3 * interval '1 hour' - interval '1 year') date_start_end,
				    		case 
				            	when date(promo.start_ + 3 * interval '1 hour') < '2016-01-01' then date(promo.end_ + 3 * interval '1 hour' - interval '1 year')
				            	else date(promo.start_ + 3 * interval '1 hour')

In [111]:
df = pd.DataFrame(data).copy()

In [112]:
df.head()

,promo_id,product_name,normal_start,date_end,app_id,app_finished
0,32961,Капитал Здоровья. Индивидуальный,2018-02-20,2018-06-01,3929,2018-03-12
1,32961,Капитал Здоровья. Индивидуальный,2018-02-20,2018-06-01,4374,2018-03-27
2,32961,Капитал Здоровья. Индивидуальный,2018-02-20,2018-06-01,4373,2018-03-27
3,32961,Капитал Здоровья. Индивидуальный,2018-02-20,2018-06-01,4868,2018-04-11
4,33234,Капитал Здоровья. Индивидуальный,2018-03-01,2018-06-01,12816,2018-05-03


In [113]:
#df['OrderPeriod'] = df.app_finished.apply(lambda x: x.strftime('%Y-%m'))
#df.head()

In [114]:
df.set_index('promo_id', inplace=True)

df['start'] = df.groupby(level=0)['normal_start'].min().apply(lambda x: x.strftime('%Y-%m'))
df.reset_index(inplace=True)

In [115]:
df['year-month-app'] = df.app_finished.apply(lambda x: x.strftime('%Y-%m'))

In [80]:
a = datetime.datetime(int(df.loc[1]['start'].split('-')[0]), int(df.loc[1]['start'].split('-')[1]), 1)

In [81]:
b = datetime.datetime(int(df.loc[1]['year-month-app'].split('-')[0]), int(df.loc[1]['year-month-app'].split('-')[1]), 1)

In [84]:
r = relativedelta(b, a)

In [88]:
r.months

int

In [158]:
df['cohort_period'] = ''
for i in range(df.shape[0]):
    a = datetime.datetime(int(df.loc[i]['start'].split('-')[0]), int(df.loc[i]['start'].split('-')[1]), 1)
    b = datetime.datetime(int(df.loc[i]['year-month-app'].split('-')[0]), int(df.loc[i]['year-month-app'].split('-')[1]), 1)
    r = relativedelta(b, a)
    
    df['cohort_period'][i] = abs(r.months) + 1

In [159]:
#df['cohort_period'] = df['year-moth-app'] - df['start']

In [160]:
df.head(20)

,promo_id,product_name,normal_start,date_end,app_id,app_finished,start,year-month-app,cohort_period
0,32961,Капитал Здоровья. Индивидуальный,2018-02-20,2018-06-01,3929,2018-03-12,2018-02,2018-03,2
1,32961,Капитал Здоровья. Индивидуальный,2018-02-20,2018-06-01,4374,2018-03-27,2018-02,2018-03,2
2,32961,Капитал Здоровья. Индивидуальный,2018-02-20,2018-06-01,4373,2018-03-27,2018-02,2018-03,2
3,32961,Капитал Здоровья. Индивидуальный,2018-02-20,2018-06-01,4868,2018-04-11,2018-02,2018-04,3
4,33234,Капитал Здоровья. Индивидуальный,2018-03-01,2018-06-01,12816,2018-05-03,2018-03,2018-05,3
5,33242,Эрго. Все врачи,2018-03-02,2018-06-30,3729,2018-03-05,2018-03,2018-03,1
6,33504,Капитал Здоровья. Семейный,2018-03-14,2018-06-01,14366,2018-06-07,2018-03,2018-06,4
7,33815,Капитал Здоровья. Индивидуальный,2018-03-15,2018-06-01,4368,2018-03-27,2018-03,2018-03,1
8,33829,Капитал Здоровья. Семейный,2018-03-16,2018-06-01,4039,2018-03-16,2018-03,2018-03,1
9,33829,Капитал Здоровья. Семейный,2018-03-16,2018-06-01,4038,2018-03-16,2018-03,2018-03,1


In [161]:
cohorts = df.groupby(['product_name', 'cohort_period'])['app_id'].count().reset_index()

In [162]:
cohorts.rename(columns={'app_id': 'consultations'
                        }, inplace=True)

In [163]:
#cohorts.set_index('product_name', inplace=True)

In [164]:
cohorts.head(20)

,product_name,cohort_period,consultations
0,VIP обследование для мужчин,2,1
1,VIP-обследование для женщин,1,7
2,VIP-обследование для женщин,2,12
3,VIP-обследование для женщин,3,22
4,VIP-обследование для женщин,4,22
5,VIP-обследование для женщин,5,11
6,VIP-обследование для женщин,6,10
7,VIP-обследование для женщин,7,9
8,VIP-обследование для женщин,8,2
9,VIP-обследование для женщин,9,5


In [165]:
cohorts.cohort_period.unique()

array([ 2,  1,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12])

In [166]:
periods = ['total', 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]

In [184]:
d = dict()

In [185]:
for i in range(cohorts.shape[0]):
    d[cohorts['product_name'][i]] = d.get(cohorts['product_name'][i], [0 for period in periods]) 
    d[cohorts['product_name'][i]][periods.index(cohorts['cohort_period'][i])] += cohorts['consultations'][i]
    d[cohorts['product_name'][i]][0] += cohorts['consultations'][i]

In [186]:
#cohorts['cohort_period'][1]

In [187]:
#abs(periods.index(cohorts['cohort_period'][12]))

In [188]:
report = pd.DataFrame.from_dict(d, orient='index').sort_index()

In [190]:
report.columns = periods

In [197]:
report.sort_values('total', ascending=False).head(25)

,total,1,2,3,4,5,6,7,8,9,10,11,12
Телемедицина_ВЭБ,29041,143,21432,7466,0,0,0,0,0,0,0,0,0
РГС Телемедицина ФЛ,16461,1446,2554,2096,1921,1536,1349,1275,996,900,836,758,794
СБСЖ_Премьер_2018_06_16,11088,1006,1426,1248,1200,1057,930,916,957,657,593,575,523
Телемедицина в ДМС Ренессанс,10726,3568,1233,861,711,645,577,573,563,527,488,548,432
ДМС_Сбербанк,6629,555,657,848,667,980,1111,909,294,143,284,158,23
Добровольное страхование,6248,408,325,560,707,701,678,705,563,478,395,396,332
РГС Телемедицина ФЛ - семейная,3797,317,614,549,420,407,350,302,229,196,146,128,139
Телемедицина_ДТ_Факт,1599,1095,191,17,6,7,5,241,34,0,0,2,1
РГС Телемедицина ЮЛ,1360,291,362,122,101,99,51,63,64,46,55,52,54
Медицинская защита к ипотеке 1,1282,304,352,233,166,137,62,22,5,1,0,0,0


In [199]:
report_percent = report.copy()

In [200]:
for i in range(1, 13):
    report_percent[i] = round(report_percent[i] / report_percent['total'] * 100, 2)

In [201]:
report_percent.sort_values('total', ascending=False).head(25)

,total,1,2,3,4,5,6,7,8,9,10,11,12
Телемедицина_ВЭБ,29041,0.49,73.80,25.71,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
РГС Телемедицина ФЛ,16461,8.78,15.52,12.73,11.67,9.33,8.20,7.75,6.05,5.47,5.08,4.60,4.82
СБСЖ_Премьер_2018_06_16,11088,9.07,12.86,11.26,10.82,9.53,8.39,8.26,8.63,5.93,5.35,5.19,4.72
Телемедицина в ДМС Ренессанс,10726,33.26,11.50,8.03,6.63,6.01,5.38,5.34,5.25,4.91,4.55,5.11,4.03
ДМС_Сбербанк,6629,8.37,9.91,12.79,10.06,14.78,16.76,13.71,4.44,2.16,4.28,2.38,0.35
Добровольное страхование,6248,6.53,5.20,8.96,11.32,11.22,10.85,11.28,9.01,7.65,6.32,6.34,5.31
РГС Телемедицина ФЛ - семейная,3797,8.35,16.17,14.46,11.06,10.72,9.22,7.95,6.03,5.16,3.85,3.37,3.66
Телемедицина_ДТ_Факт,1599,68.48,11.94,1.06,0.38,0.44,0.31,15.07,2.13,0.00,0.00,0.13,0.06
РГС Телемедицина ЮЛ,1360,21.40,26.62,8.97,7.43,7.28,3.75,4.63,4.71,3.38,4.04,3.82,3.97
Медицинская защита к ипотеке 1,1282,23.71,27.46,18.17,12.95,10.69,4.84,1.72,0.39,0.08,0.00,0.00,0.00
